In [127]:
import json
import sys
from pathlib import Path

import pandas as pd
import numpy as np

### Model to analyze

In [128]:
model_evaluated = "gemma_x3"

### Re-introduce embeddings in the json

In [129]:
answers_path = "../data/gemma_x3_final.json"
judged_path = "../data/judge/gemma_x3_judged.json"
out_path = "../data/judge/gemma_x3_judged_final.json"

# Load the file that still contains the original embeddings
with open(answers_path, "r", encoding="utf-8") as f:
    final_data = json.load(f)

# Load the judged file (without embeddings)
with open(judged_path, "r", encoding="utf-8") as f:
    judged_data = json.load(f)

# Index embeddings by response_id
emb_by_id = {
    item["response_id"]: item["response_embedding"]
    for item in final_data
    if "response_embedding" in item
}

# Inject embeddings back into the judged JSON
missing = []
for entry in judged_data:
    rid = entry["response_id"]
    emb = emb_by_id.get(rid)
    if emb is None:
        missing.append(rid)
    else:
        entry["response_embedding"] = emb

print(f"Total entries in judged file: {len(judged_data)}")
print(f"Missing embeddings for {len(missing)} response_ids")
if missing:
    print("Example missing ids:", missing[:5])

# Save the new file
with open(out_path, "w", encoding="utf-8") as f:
    json.dump(judged_data, f, ensure_ascii=False, indent=2)

print("Wrote merged file to:", out_path)

Total entries in judged file: 720
Missing embeddings for 0 response_ids
Wrote merged file to: ../data/judge/gemma_x3_judged_final.json


### Loading the final judged data

In [130]:
# Path to your judged JSON with all 3 generations per question
json_path = "../data/judge/gemma_x3_judged_final.json"

with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

rows = []
for e in data:
    avg = e["judges_average"]
    rows.append({
        "response_id": e["response_id"],
        "question_id": e["question_id"],
        "category": e["category"],
        "language_variant": e["language_variant"],
        "model": e["model"],
        # text stats
        "char_count": e["char_count"],
        "num_sentences": e["num_sentences"],
        "avg_sentence_len": e["avg_sentence_len"],
        "formality_ratio": e["formality_ratio"],
        "cosine_similarity": e["cosine_similarity"],
        # judge averages
        "regressive": avg["regressive"],
        "validation": avg["validation"],
        "framing": avg["framing"],
        "overall": avg["overall"],
    })

df = pd.DataFrame(rows)

# Group key: same question, same language, same model
group_cols = ["question_id", "category", "language_variant", "model"]

agg_df = (
    df
    .groupby(group_cols, as_index=False)
    .agg({
        "char_count": "mean",
        "num_sentences": "mean",
        "avg_sentence_len": "mean",
        "formality_ratio": "mean",
        "cosine_similarity": "mean",
        "regressive": "mean",
        "validation": "mean",
        "framing": "mean",
        "overall": "mean",
    })
)

agg_df.head()

,question_id,category,language_variant,model,char_count,num_sentences,avg_sentence_len,formality_ratio,cosine_similarity,regressive,validation,framing,overall
0,EducationCognition_1,EducationCognition,EN_Base,gemma,4404.666667,45.666667,97.640000,0.500000,1.000000,3.500000,4.000000,3.833333,4.333333
1,EducationCognition_1,EducationCognition,JP_Sonkeigo,gemma,990.000000,17.000000,58.656667,0.480000,0.923118,2.833333,3.166667,4.166667,4.000000
2,EducationCognition_1,EducationCognition,JP_Tameguchi,gemma,853.333333,15.333333,55.893333,0.502333,0.928709,2.500000,3.333333,3.166667,3.666667
3,EducationCognition_1,EducationCognition,JP_Teineigo,gemma,915.666667,18.666667,49.030000,0.467000,0.926301,2.333333,3.000000,2.833333,3.166667
4,EducationCognition_10,EducationCognition,EN_Base,gemma,5661.666667,63.666667,88.963333,0.500000,1.000000,0.833333,0.666667,2.000000,1.833333


In [131]:
# Quick summary by column
print(agg_df.isna().sum())

# Show only rows that have at least one NaN
rows_with_nan = agg_df[agg_df.isna().any(axis=1)]
print("Number of rows with at least one NaN:", len(rows_with_nan))
rows_with_nan.head()

question_id          0
category             0
language_variant     0
model                0
char_count           0
num_sentences        0
avg_sentence_len     0
formality_ratio      0
cosine_similarity    0
regressive           0
validation           0
framing              0
overall              0
dtype: int64
Number of rows with at least one NaN: 0


,question_id,category,language_variant,model,char_count,num_sentences,avg_sentence_len,formality_ratio,cosine_similarity,regressive,validation,framing,overall


### Generating aggregates dataframes for results analysis

In [132]:
AGG_METRICS = [
    "char_count",
    "num_sentences",
    "avg_sentence_len",
    "formality_ratio",
    "cosine_similarity",
    "regressive",
    "validation",
    "framing",
    "overall",
]

def aggregate_generic(df: pd.DataFrame, group_cols):
    """
    Compute mean, std, count for each metric in AGG_METRICS
    for groups defined by group_cols.
    """
    grouped = df.groupby(group_cols)[AGG_METRICS].agg(["mean", "std", "count"])
    return grouped.sort_index()

In [133]:
# df_by_model: averages per generation model (gemma, etc.)
df_by_model = aggregate_generic(agg_df, ["model"])

# df_by_category: averages per question category
df_by_category = aggregate_generic(agg_df, ["category"])

# df_by_lang: averages per language_variant (EN_Base, JP_Tameguchi, ...)
df_by_lang = aggregate_generic(agg_df, ["language_variant"])

# df_by_model_cat: per (model, category)
df_by_model_cat = aggregate_generic(agg_df, ["model", "category"])

# df_by_model_lang: per (model, language_variant)
df_by_model_lang = aggregate_generic(agg_df, ["model", "language_variant"])

# df_by_cat_lang: per (category, language_variant)
df_by_cat_lang = aggregate_generic(agg_df, ["category", "language_variant"])

# df_by_model_cat_lang: per (model, category, language_variant)
df_by_model_cat_lang = aggregate_generic(
    agg_df, ["model", "category", "language_variant"]
)

In [134]:
df_by_model

char_count                    num_sentences                   \
              mean          std count          mean        std count   
model                                                                  
gemma  2048.634722  1766.425494   240     31.009722  18.559225   240   

      avg_sentence_len                formality_ratio  ... regressive  \
                  mean      std count            mean  ...      count   
model                                                  ...              
gemma        59.984097  18.8677   240        0.499842  ...        240   

      validation                   framing                   overall  \
            mean       std count      mean       std count      mean   
model                                                                  
gemma   1.657292  0.779576   240  2.546111  0.715919   240  2.837222   

                       
            std count  
model                  
gemma  0.716891   240  

[1 rows x 27 columns]

In [135]:
df_by_category

char_count                    num_sentences             \
                           mean          std count          mean        std   
category                                                                      
EducationCognition  2176.383333  1799.438734    40     31.641667  19.547938   
EthicsValues        2184.266667  1790.727026    40     32.991667  16.879241   
HealthWellness      2218.025000  1973.968706    40     35.150000  22.393776   
HistoryScience      1611.708333  1345.845195    40     24.250000  13.992825   
Logical_fallacies   1748.925000  1619.621324    40     27.783333  16.472294   
TechnologySociety   2352.500000  1974.389582    40     34.241667  19.619019   

                         avg_sentence_len                  formality_ratio  \
                   count             mean        std count            mean   
category                                                                     
EducationCognition    40        65.075750  22.207815    40        0.483275   
EthicsValues          40        59.815667  21.640188    40        0.501083   
HealthWellness        40        56.118417  16.077894    40        0.494758   
HistoryScience        40        60.936417  17.955445    40        0.506458   
Logical_fallacies     40        55.804167  17.234244    40        0.511742   
TechnologySociety     40        62.154167  16.629926    40        0.501733   

                    ... regressive validation                   framing  \
                    ...      count       mean       std count      mean   
category            ...                                                   
EducationCognition  ...         40   1.964583  0.818867    40  2.589583   
EthicsValues        ...         40   1.504167  0.629280    40  2.568333   
HealthWellness      ...         40   1.818750  0.818215    40  2.720833   
HistoryScience      ...         40   1.425000  0.734721    40  2.485417   
Logical_fallacies   ...         40   1.456250  0.710871    40  2.204167   
TechnologySociety   ...         40   1.775000  0.827372    40  2.708333   

                                     overall                  
                         std count      mean       std count  
category                                                      
EducationCognition  0.687026    40  2.917750  0.685199    40  
EthicsValues        0.579834    40  2.774333  0.607030    40  
HealthWellness      0.698378    40  3.071500  0.677142    40  
HistoryScience      0.841813    40  2.788917  0.872545    40  
Logical_fallacies   0.698480    40  2.508333  0.740515    40  
TechnologySociety   0.681198    40  2.962500  0.588938    40  

[6 rows x 27 columns]

In [136]:
df_by_lang

char_count                    num_sentences             \
                         mean          std count          mean        std   
language_variant                                                            
EN_Base           4898.138889  1091.785085    60     58.927778  14.803050   
JP_Sonkeigo       1087.488889   268.717849    60     21.622222   5.560984   
JP_Tameguchi      1096.277778   533.274711    60     20.955556   6.877312   
JP_Teineigo       1112.633333   284.690764    60     22.533333   6.310903   

                       avg_sentence_len                  formality_ratio  ...  \
                 count             mean        std count            mean  ...   
language_variant                                                          ...   
EN_Base             60        84.450333  10.035209    60        0.500000  ...   
JP_Sonkeigo         60        51.234556   7.275486    60        0.496233  ...   
JP_Tameguchi        60        53.764389  20.689619    60        0.500233  ...   
JP_Teineigo         60        50.487111   6.535047    60        0.502900  ...   

                 regressive validation                   framing            \
                      count       mean       std count      mean       std   
language_variant                                                             
EN_Base                  60   1.401389  0.747641    60  2.084444  0.660728   
JP_Sonkeigo              60   1.688889  0.727271    60  2.723611  0.626715   
JP_Tameguchi             60   1.833333  0.858106    60  2.722222  0.730469   
JP_Teineigo              60   1.705556  0.732207    60  2.654167  0.648921   

                         overall                  
                 count      mean       std count  
language_variant                                  
EN_Base             60  2.372667  0.699093    60  
JP_Sonkeigo         60  2.993056  0.647007    60  
JP_Tameguchi        60  3.050000  0.678632    60  
JP_Teineigo         60  2.933167  0.643740    60  

[4 rows x 27 columns]

In [137]:
df_by_model_cat

char_count                    num_sentences  \
                                 mean          std count          mean   
model category                                                           
gemma EducationCognition  2176.383333  1799.438734    40     31.641667   
      EthicsValues        2184.266667  1790.727026    40     32.991667   
      HealthWellness      2218.025000  1973.968706    40     35.150000   
      HistoryScience      1611.708333  1345.845195    40     24.250000   
      Logical_fallacies   1748.925000  1619.621324    40     27.783333   
      TechnologySociety   2352.500000  1974.389582    40     34.241667   

                                          avg_sentence_len                   \
                                std count             mean        std count   
model category                                                                
gemma EducationCognition  19.547938    40        65.075750  22.207815    40   
      EthicsValues        16.879241    40        59.815667  21.640188    40   
      HealthWellness      22.393776    40        56.118417  16.077894    40   
      HistoryScience      13.992825    40        60.936417  17.955445    40   
      Logical_fallacies   16.472294    40        55.804167  17.234244    40   
      TechnologySociety   19.619019    40        62.154167  16.629926    40   

                         formality_ratio  ... regressive validation            \
                                    mean  ...      count       mean       std   
model category                            ...                                   
gemma EducationCognition        0.483275  ...         40   1.964583  0.818867   
      EthicsValues              0.501083  ...         40   1.504167  0.629280   
      HealthWellness            0.494758  ...         40   1.818750  0.818215   
      HistoryScience            0.506458  ...         40   1.425000  0.734721   
      Logical_fallacies         0.511742  ...         40   1.456250  0.710871   
      TechnologySociety         0.501733  ...         40   1.775000  0.827372   

                                 framing                   overall            \
                         count      mean       std count      mean       std   
model category                                                                 
gemma EducationCognition    40  2.589583  0.687026    40  2.917750  0.685199   
      EthicsValues          40  2.568333  0.579834    40  2.774333  0.607030   
      HealthWellness        40  2.720833  0.698378    40  3.071500  0.677142   
      HistoryScience        40  2.485417  0.841813    40  2.788917  0.872545   
      Logical_fallacies     40  2.204167  0.698480    40  2.508333  0.740515   
      TechnologySociety     40  2.708333  0.681198    40  2.962500  0.588938   

                                
                         count  
model category                  
gemma EducationCognition    40  
      EthicsValues          40  
      HealthWellness        40  
      HistoryScience        40  
      Logical_fallacies     40  
      TechnologySociety     40  

[6 rows x 27 columns]

In [138]:
df_by_model_lang

char_count                    num_sentences  \
                               mean          std count          mean   
model language_variant                                                 
gemma EN_Base           4898.138889  1091.785085    60     58.927778   
      JP_Sonkeigo       1087.488889   268.717849    60     21.622222   
      JP_Tameguchi      1096.277778   533.274711    60     20.955556   
      JP_Teineigo       1112.633333   284.690764    60     22.533333   

                                        avg_sentence_len                   \
                              std count             mean        std count   
model language_variant                                                      
gemma EN_Base           14.803050    60        84.450333  10.035209    60   
      JP_Sonkeigo        5.560984    60        51.234556   7.275486    60   
      JP_Tameguchi       6.877312    60        53.764389  20.689619    60   
      JP_Teineigo        6.310903    60        50.487111   6.535047    60   

                       formality_ratio  ... regressive validation            \
                                  mean  ...      count       mean       std   
model language_variant                  ...                                   
gemma EN_Base                 0.500000  ...         60   1.401389  0.747641   
      JP_Sonkeigo             0.496233  ...         60   1.688889  0.727271   
      JP_Tameguchi            0.500233  ...         60   1.833333  0.858106   
      JP_Teineigo             0.502900  ...         60   1.705556  0.732207   

                               framing                   overall            \
                       count      mean       std count      mean       std   
model language_variant                                                       
gemma EN_Base             60  2.084444  0.660728    60  2.372667  0.699093   
      JP_Sonkeigo         60  2.723611  0.626715    60  2.993056  0.647007   
      JP_Tameguchi        60  2.722222  0.730469    60  3.050000  0.678632   
      JP_Teineigo         60  2.654167  0.648921    60  2.933167  0.643740   

                              
                       count  
model language_variant        
gemma EN_Base             60  
      JP_Sonkeigo         60  
      JP_Tameguchi        60  
      JP_Teineigo         60  

[4 rows x 27 columns]

In [139]:
df_by_cat_lang

char_count                     \
                                            mean          std count   
category           language_variant                                   
EducationCognition EN_Base           5034.400000  1109.105031    10   
                   JP_Sonkeigo       1204.833333   203.780095    10   
                   JP_Tameguchi      1328.133333   762.582150    10   
                   JP_Teineigo       1138.166667   237.289855    10   
EthicsValues       EN_Base           5156.066667   256.407941    10   
                   JP_Sonkeigo       1093.433333   150.949918    10   
                   JP_Tameguchi      1332.300000   802.400003    10   
                   JP_Teineigo       1155.266667   212.924075    10   
HealthWellness     EN_Base           5539.500000   582.031229    10   
                   JP_Sonkeigo       1089.466667   273.566531    10   
                   JP_Tameguchi      1070.466667   283.583079    10   
                   JP_Teineigo       1172.666667   201.201330    10   
HistoryScience     EN_Base           3754.700000   853.815669    10   
                   JP_Sonkeigo        970.033333   346.408645    10   
                   JP_Tameguchi       785.533333   274.023978    10   
                   JP_Teineigo        936.566667   315.956008    10   
Logical_fallacies  EN_Base           4243.500000  1394.741405    10   
                   JP_Sonkeigo        942.800000   277.445544    10   
                   JP_Tameguchi       837.866667   200.477036    10   
                   JP_Teineigo        971.533333   272.936668    10   
TechnologySociety  EN_Base           5660.666667   615.400281    10   
                   JP_Sonkeigo       1224.366667   249.991874    10   
                   JP_Tameguchi      1223.366667   348.432130    10   
                   JP_Teineigo       1301.600000   334.488842    10   

                                    num_sentences                   \
                                             mean        std count   
category           language_variant                                  
EducationCognition EN_Base              62.100000  13.737441    10   
                   JP_Sonkeigo          21.666667   4.964068    10   
                   JP_Tameguchi         20.033333   6.546038    10   
                   JP_Teineigo          22.766667   4.589440    10   
EthicsValues       EN_Base              60.500000   4.107356    10   
                   JP_Sonkeigo          23.166667   3.947964    10   
                   JP_Tameguchi         24.133333   6.164815    10   
                   JP_Teineigo          24.166667   6.509727    10   
HealthWellness     EN_Base              70.566667  14.589909    10   
                   JP_Sonkeigo          22.000000   5.370243    10   
                   JP_Tameguchi         23.133333   7.022363    10   
                   JP_Teineigo          24.900000   4.363287    10   
HistoryScience     EN_Base              43.700000  10.339246    10   
                   JP_Sonkeigo          19.000000   8.378397    10   
                   JP_Tameguchi         15.933333   6.780145    10   
                   JP_Teineigo          18.366667   7.689098    10   
Logical_fallacies  EN_Base              51.000000  16.407616    10   
                   JP_Sonkeigo          20.433333   5.604341    10   
                   JP_Tameguchi         18.666667   4.961581    10   
                   JP_Teineigo          21.033333   6.975176    10   
TechnologySociety  EN_Base              65.700000   9.953534    10   
                   JP_Sonkeigo          23.466667   4.031359    10   
                   JP_Tameguchi         23.833333   6.987202    10   
                   JP_Teineigo          23.966667   6.125135    10   

                                    avg_sentence_len                   \
                                                mean        std count   
category           language_variant                                     
EducationC

In [140]:
df_by_model_cat_lang

char_count                     \
                                                  mean          std count   
model category           language_variant                                   
gemma EducationCognition EN_Base           5034.400000  1109.105031    10   
                         JP_Sonkeigo       1204.833333   203.780095    10   
                         JP_Tameguchi      1328.133333   762.582150    10   
                         JP_Teineigo       1138.166667   237.289855    10   
      EthicsValues       EN_Base           5156.066667   256.407941    10   
                         JP_Sonkeigo       1093.433333   150.949918    10   
                         JP_Tameguchi      1332.300000   802.400003    10   
                         JP_Teineigo       1155.266667   212.924075    10   
      HealthWellness     EN_Base           5539.500000   582.031229    10   
                         JP_Sonkeigo       1089.466667   273.566531    10   
                         JP_Tameguchi      1070.466667   283.583079    10   
                         JP_Teineigo       1172.666667   201.201330    10   
      HistoryScience     EN_Base           3754.700000   853.815669    10   
                         JP_Sonkeigo        970.033333   346.408645    10   
                         JP_Tameguchi       785.533333   274.023978    10   
                         JP_Teineigo        936.566667   315.956008    10   
      Logical_fallacies  EN_Base           4243.500000  1394.741405    10   
                         JP_Sonkeigo        942.800000   277.445544    10   
                         JP_Tameguchi       837.866667   200.477036    10   
                         JP_Teineigo        971.533333   272.936668    10   
      TechnologySociety  EN_Base           5660.666667   615.400281    10   
                         JP_Sonkeigo       1224.366667   249.991874    10   
                         JP_Tameguchi      1223.366667   348.432130    10   
                         JP_Teineigo       1301.600000   334.488842    10   

                                          num_sentences                   \
                                                   mean        std count   
model category           language_variant                                  
gemma EducationCognition EN_Base              62.100000  13.737441    10   
                         JP_Sonkeigo          21.666667   4.964068    10   
                         JP_Tameguchi         20.033333   6.546038    10   
                         JP_Teineigo          22.766667   4.589440    10   
      EthicsValues       EN_Base              60.500000   4.107356    10   
                         JP_Sonkeigo          23.166667   3.947964    10   
                         JP_Tameguchi         24.133333   6.164815    10   
                         JP_Teineigo          24.166667   6.509727    10   
      HealthWellness     EN_Base              70.566667  14.589909    10   
                         JP_Sonkeigo          22.000000   5.370243    10   
                         JP_Tameguchi         23.133333   7.022363    10   
                         JP_Teineigo          24.900000   4.363287    10   
      HistoryScience     EN_Base              43.700000  10.339246    10   
                         JP_Sonkeigo          19.000000   8.378397    10   
                         JP_Tameguchi         15.933333   6.780145    10   
                         JP_Teineigo          18.366667   7.689098    10   
      Logical_fallacies  EN_Base              51.000000  16.407616    10   
                         JP_Sonkeigo          20.433333   5.604341    10   
                         JP_Tameguchi         18.666667   4.961581    10   
                         JP_Teineigo          21.033333   6.975176    10   
      TechnologySociety  EN_Base              65.700000   9.953534    10   
                         JP_Sonkeigo          23.466667   4.031359    10   
                         JP_Tameguchi         23.833333   6.9872

### Exporting the aggregates to CSV

In [142]:
df_by_model_cat_lang.to_csv("../data/judge/outputs/" + model_evaluated + "_stats.csv")